In [384]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [385]:
df = pd.read_csv('//content/drive/MyDrive/Colab Notebooks/fuentes/transactions.csv')

In [386]:
df

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,1102483,2018-07-25 00:00:29,1111,2328,40.77,9936029,115,0,0,...,24.0,29.191250,105.0,30.141810,1.0,0.0,10.0,0.0,32.0,0.0
1,1,1102484,2018-07-25 00:01:08,676,6846,9.62,9936068,115,0,0,...,15.0,19.151333,61.0,18.253607,2.0,0.0,5.0,0.0,27.0,0.0
2,2,1102485,2018-07-25 00:01:35,402,4771,81.55,9936095,115,0,0,...,11.0,183.875455,50.0,112.001800,0.0,0.0,12.0,0.0,37.0,0.0
3,3,1102486,2018-07-25 00:01:43,4218,863,23.10,9936103,115,0,0,...,24.0,21.201667,93.0,19.987849,2.0,0.0,6.0,0.0,22.0,0.0
4,4,1102487,2018-07-25 00:02:26,3711,3599,59.25,9936146,115,0,0,...,27.0,80.427407,110.0,74.158455,1.0,0.0,10.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,201290,1303773,2018-08-14 23:57:03,460,6133,16.72,11750223,135,0,0,...,7.0,19.210000,21.0,22.987619,1.0,0.0,3.0,0.0,20.0,0.0
201291,201291,1303774,2018-08-14 23:58:24,3101,3229,38.16,11750304,135,0,0,...,28.0,46.151429,97.0,45.018144,2.0,0.0,13.0,0.0,42.0,0.0
201292,201292,1303775,2018-08-14 23:58:24,4783,7511,69.85,11750304,135,0,0,...,24.0,95.770417,101.0,89.886040,0.0,0.0,3.0,0.0,34.0,0.0
201293,201293,1303776,2018-08-14 23:58:45,2883,8550,14.99,11750325,135,0,0,...,19.0,18.450000,77.0,15.848052,3.0,0.0,7.0,0.0,20.0,0.0


# 1) Ingeniería de Atributos

###Preprocesamiento

In [387]:
df_copia = df.drop(['Unnamed: 0'], axis = 1)

In [388]:
df_copia

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,1102483,2018-07-25 00:00:29,1111,2328,40.77,9936029,115,0,0,0,...,24.0,29.191250,105.0,30.141810,1.0,0.0,10.0,0.0,32.0,0.0
1,1102484,2018-07-25 00:01:08,676,6846,9.62,9936068,115,0,0,0,...,15.0,19.151333,61.0,18.253607,2.0,0.0,5.0,0.0,27.0,0.0
2,1102485,2018-07-25 00:01:35,402,4771,81.55,9936095,115,0,0,0,...,11.0,183.875455,50.0,112.001800,0.0,0.0,12.0,0.0,37.0,0.0
3,1102486,2018-07-25 00:01:43,4218,863,23.10,9936103,115,0,0,0,...,24.0,21.201667,93.0,19.987849,2.0,0.0,6.0,0.0,22.0,0.0
4,1102487,2018-07-25 00:02:26,3711,3599,59.25,9936146,115,0,0,0,...,27.0,80.427407,110.0,74.158455,1.0,0.0,10.0,0.0,38.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,1303773,2018-08-14 23:57:03,460,6133,16.72,11750223,135,0,0,0,...,7.0,19.210000,21.0,22.987619,1.0,0.0,3.0,0.0,20.0,0.0
201291,1303774,2018-08-14 23:58:24,3101,3229,38.16,11750304,135,0,0,0,...,28.0,46.151429,97.0,45.018144,2.0,0.0,13.0,0.0,42.0,0.0
201292,1303775,2018-08-14 23:58:24,4783,7511,69.85,11750304,135,0,0,0,...,24.0,95.770417,101.0,89.886040,0.0,0.0,3.0,0.0,34.0,0.0
201293,1303776,2018-08-14 23:58:45,2883,8550,14.99,11750325,135,0,0,0,...,19.0,18.450000,77.0,15.848052,3.0,0.0,7.0,0.0,20.0,0.0


Se elimina la columna "Unnamed: 0".

In [389]:
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201295 entries, 0 to 201294
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   TRANSACTION_ID                       201295 non-null  int64  
 1   TX_DATETIME                          201295 non-null  object 
 2   CUSTOMER_ID                          201295 non-null  int64  
 3   TERMINAL_ID                          201295 non-null  int64  
 4   TX_AMOUNT                            201295 non-null  float64
 5   TX_TIME_SECONDS                      201295 non-null  int64  
 6   TX_TIME_DAYS                         201295 non-null  int64  
 7   TX_FRAUD                             201295 non-null  int64  
 8   TX_FRAUD_SCENARIO                    201295 non-null  int64  
 9   TX_DURING_WEEKEND                    201295 non-null  int64  
 10  TX_DURING_NIGHT                      201295 non-null  int64  
 11  CUSTOMER_ID_N

No tiene valores nulos.

In [390]:
df_copia['TRANSACTION_ID'].duplicated().sum()

0

No tiene valores duplicados.

El análisis de outliers tiene sentido hacerlo únicamente sobre los datos de los montos de las transacciones.

In [391]:
Q1 = df_copia['TX_AMOUNT'].quantile(0.25)
Q3 = df_copia['TX_AMOUNT'].quantile(0.75)
IQR = Q3 - Q1
outliers_monto = df_copia[((df_copia['TX_AMOUNT'] < (Q1 - 1.5 * IQR)) | (df_copia['TX_AMOUNT'] > (Q3 + 1.5 * IQR)))].index

In [392]:
outliers_monto

Int64Index([    42,     86,     93,    115,    292,    399,    429,    506,
               565,    584,
            ...
            200847, 200870, 200882, 201023, 201030, 201113, 201155, 201239,
            201250, 201288],
           dtype='int64', length=3104)

Hay 3104 datos que se consideran outliers, pero no se eliminarán ya que es importante considerarlos dentro del entrenamiento de modelo porque que son posibles casos de fraude.

In [393]:
y = df_copia.TX_FRAUD
df_copia = df_copia.drop(['TX_FRAUD'], axis = 1)

In [394]:
y.unique()

array([0, 1])

Se separa del conjunto de datos la variable dependiente, que en este caso es "TX_FRAUD", porque es la variable sobre la cual se trabajará y entrenará el modelo.

Todas las columnas, excepto 'TX_DATATIME', tienen un tipo de dato numérico, por lo cual, no es necesario aplicar ningún tipo de codificación de variables categóricas.

Para poder estandarizar la fecha lo que se hará es crear nuevas variables en donde estarán guardados el día de la semana, mes y año.



In [395]:
df_copia['TX_DATETIME'] = pd.to_datetime(df_copia['TX_DATETIME'])

In [396]:
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201295 entries, 0 to 201294
Data columns (total 22 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   TRANSACTION_ID                       201295 non-null  int64         
 1   TX_DATETIME                          201295 non-null  datetime64[ns]
 2   CUSTOMER_ID                          201295 non-null  int64         
 3   TERMINAL_ID                          201295 non-null  int64         
 4   TX_AMOUNT                            201295 non-null  float64       
 5   TX_TIME_SECONDS                      201295 non-null  int64         
 6   TX_TIME_DAYS                         201295 non-null  int64         
 7   TX_FRAUD_SCENARIO                    201295 non-null  int64         
 8   TX_DURING_WEEKEND                    201295 non-null  int64         
 9   TX_DURING_NIGHT                      201295 non-null  int64         
 

In [397]:
df_copia['dia_semana'] = df_copia['TX_DATETIME'].dt.dayofweek
df_copia['mes'] = df_copia['TX_DATETIME'].dt.month
df_copia['anio'] = df_copia['TX_DATETIME'].dt.year

In [398]:
df_copia

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW,dia_semana,mes,anio
0,1102483,2018-07-25 00:00:29,1111,2328,40.77,9936029,115,0,0,1,...,30.141810,1.0,0.0,10.0,0.0,32.0,0.0,2,7,2018
1,1102484,2018-07-25 00:01:08,676,6846,9.62,9936068,115,0,0,1,...,18.253607,2.0,0.0,5.0,0.0,27.0,0.0,2,7,2018
2,1102485,2018-07-25 00:01:35,402,4771,81.55,9936095,115,0,0,1,...,112.001800,0.0,0.0,12.0,0.0,37.0,0.0,2,7,2018
3,1102486,2018-07-25 00:01:43,4218,863,23.10,9936103,115,0,0,1,...,19.987849,2.0,0.0,6.0,0.0,22.0,0.0,2,7,2018
4,1102487,2018-07-25 00:02:26,3711,3599,59.25,9936146,115,0,0,1,...,74.158455,1.0,0.0,10.0,0.0,38.0,0.0,2,7,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,1303773,2018-08-14 23:57:03,460,6133,16.72,11750223,135,0,0,0,...,22.987619,1.0,0.0,3.0,0.0,20.0,0.0,1,8,2018
201291,1303774,2018-08-14 23:58:24,3101,3229,38.16,11750304,135,0,0,0,...,45.018144,2.0,0.0,13.0,0.0,42.0,0.0,1,8,2018
201292,1303775,2018-08-14 23:58:24,4783,7511,69.85,11750304,135,0,0,0,...,89.886040,0.0,0.0,3.0,0.0,34.0,0.0,1,8,2018
201293,1303776,2018-08-14 23:58:45,2883,8550,14.99,11750325,135,0,0,0,...,15.848052,3.0,0.0,7.0,0.0,20.0,0.0,1,8,2018


In [399]:
df_copia = df_copia.drop(['TX_DATETIME'], axis = 1)

In [400]:
df_copia

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW,dia_semana,mes,anio
0,1102483,1111,2328,40.77,9936029,115,0,0,1,2.0,...,30.141810,1.0,0.0,10.0,0.0,32.0,0.0,2,7,2018
1,1102484,676,6846,9.62,9936068,115,0,0,1,4.0,...,18.253607,2.0,0.0,5.0,0.0,27.0,0.0,2,7,2018
2,1102485,402,4771,81.55,9936095,115,0,0,1,6.0,...,112.001800,0.0,0.0,12.0,0.0,37.0,0.0,2,7,2018
3,1102486,4218,863,23.10,9936103,115,0,0,1,8.0,...,19.987849,2.0,0.0,6.0,0.0,22.0,0.0,2,7,2018
4,1102487,3711,3599,59.25,9936146,115,0,0,1,5.0,...,74.158455,1.0,0.0,10.0,0.0,38.0,0.0,2,7,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,1303773,460,6133,16.72,11750223,135,0,0,0,1.0,...,22.987619,1.0,0.0,3.0,0.0,20.0,0.0,1,8,2018
201291,1303774,3101,3229,38.16,11750304,135,0,0,0,7.0,...,45.018144,2.0,0.0,13.0,0.0,42.0,0.0,1,8,2018
201292,1303775,4783,7511,69.85,11750304,135,0,0,0,3.0,...,89.886040,0.0,0.0,3.0,0.0,34.0,0.0,1,8,2018
201293,1303776,2883,8550,14.99,11750325,135,0,0,0,3.0,...,15.848052,3.0,0.0,7.0,0.0,20.0,0.0,1,8,2018


Ahora sí se procede a la normalización de los datos utilizando StandarScaler.

In [401]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_copia_std = scaler.fit_transform(df_copia)
df_columns = df_copia.columns
df_copia_std = pd.DataFrame(df_copia_std, columns = df_columns)

Una vez con los datos normalizados se puede proceder al entrenamiento del modelo.

In [402]:
df_copia_std

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW,dia_semana,mes,anio
0,-1.732042,-0.963884,-0.920137,-0.304920,-1.732272,-1.650540,-0.092076,-0.630817,2.188454,-0.849046,...,-0.804512,-0.001464,-0.078603,0.996956,-0.118318,0.244269,-0.147354,-0.499036,-1.411977,0.0
1,-1.732025,-1.264345,0.644669,-1.045527,-1.732197,-1.650540,-0.092076,-0.630817,2.188454,0.239704,...,-1.210253,0.977715,-0.078603,-0.662409,-0.118318,-0.360128,-0.147354,-0.499036,-1.411977,0.0
2,-1.732008,-1.453601,-0.074005,0.664646,-1.732146,-1.650540,-0.092076,-0.630817,2.188454,1.328454,...,1.989342,-0.980643,-0.078603,1.660701,-0.118318,0.848665,-0.147354,-0.499036,-1.411977,0.0
3,-1.731991,1.182169,-1.427538,-0.725033,-1.732130,-1.650540,-0.092076,-0.630817,2.188454,2.417204,...,-1.151064,0.977715,-0.078603,-0.330536,-0.118318,-0.964524,-0.147354,-0.499036,-1.411977,0.0
4,-1.731973,0.831976,-0.479927,0.134452,-1.732048,-1.650540,-0.092076,-0.630817,2.188454,0.784079,...,0.697761,-0.001464,-0.078603,0.996956,-0.118318,0.969544,-0.147354,-0.499036,-1.411977,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201290,1.731973,-1.413540,0.397722,-0.876721,1.734225,1.653021,-0.092076,-0.630817,-0.456944,-1.393421,...,-1.048683,-0.001464,-0.078603,-1.326154,-0.118318,-1.206282,-0.147354,-0.999624,0.708227,0.0
201291,1.731991,0.410640,-0.608076,-0.366974,1.734380,1.653021,-0.092076,-0.630817,-0.456944,1.872829,...,-0.296788,0.977715,-0.078603,1.992574,-0.118318,1.453061,-0.147354,-0.999624,0.708227,0.0
201292,1.732008,1.572423,0.874992,0.386472,1.734380,1.653021,-0.092076,-0.630817,-0.456944,-0.304671,...,1.234538,-0.980643,-0.078603,-1.326154,-0.118318,0.486027,-0.147354,-0.999624,0.708227,0.0
201293,1.732025,0.260064,1.234849,-0.917853,1.734420,1.653021,-0.092076,-0.630817,-0.456944,-0.304671,...,-1.292354,1.956894,-0.078603,0.001337,-0.118318,-1.206282,-0.147354,-0.999624,0.708227,0.0


# 2) Entrenamiento del Modelo

El tipo de modelo predictivo que se hará es en base a un aprendizaje supervisado y la técnica que se utilizará es árbol de decisiones.

### Modelo 1

En este primer modelo el árbol de decisiones no tendrá un límite en la altura.

In [403]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
x_train1, x_test1, y_train1, y_test1 = train_test_split(df_copia_std, y, test_size = 0.3, random_state = 42)

In [404]:
x_train1

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW,dia_semana,mes,anio
41161,-1.023699,-1.342396,-1.315321,0.794698,-1.008577,-0.989828,-0.092076,1.585245,-0.456944,0.239704,...,-0.607915,-0.001464,-0.078603,-0.330536,-0.118318,1.090423,-0.147354,1.503315,-1.411977,0.0
4215,-1.659506,0.144024,0.881572,-0.668923,-1.654540,-1.650540,-0.092076,-0.630817,-0.456944,-1.393421,...,-1.272318,0.977715,-0.078603,-0.330536,-0.118318,-0.239248,-0.147354,-0.499036,-1.411977,0.0
93372,-0.125196,1.485393,1.350876,-0.292556,-0.141100,-0.163937,-0.092076,-0.630817,-0.456944,0.239704,...,-0.568909,2.936073,-0.078603,0.001337,-0.118318,0.244269,-0.147354,0.502140,0.708227,0.0
73552,-0.466280,-1.165573,-1.303199,2.048381,-0.479152,-0.494293,-0.092076,-0.630817,-0.456944,1.328454,...,0.943988,-0.980643,-0.078603,-0.330536,-0.118318,0.002510,-0.147354,-0.499036,0.708227,0.0
122945,0.383728,1.513022,-1.262676,-1.030073,0.367102,0.331597,-0.092076,-0.630817,-0.456944,-0.849046,...,-1.533419,0.977715,-0.078603,1.328828,-0.118318,1.815698,-0.147354,-1.500212,0.708227,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.330965,1.496444,0.555658,-0.373631,0.331929,0.331597,-0.092076,-0.630817,-0.456944,-0.849046,...,-0.321725,-0.001464,-0.078603,1.660701,-0.118318,-0.118369,0.419214,-1.500212,0.708227,0.0
103694,0.052436,0.122611,-1.294887,-0.237160,0.032607,0.001241,-0.092076,1.585245,-0.456944,-0.849046,...,-0.767384,-0.980643,-0.078603,1.992574,-0.118318,0.969544,-0.147354,1.002727,0.708227,0.0
131932,0.538386,-0.953523,-0.211852,-0.368876,0.523296,0.496775,-0.092076,-0.630817,-0.456944,-0.304671,...,-0.220635,-0.001464,-0.078603,-0.330536,-0.118318,1.090423,-0.147354,-0.999624,0.708227,0.0
146867,0.795404,-0.589516,0.501281,-0.589275,0.807061,0.827131,-0.092076,-0.630817,-0.456944,-1.393421,...,-0.924227,-0.001464,-0.078603,-1.658027,-0.118318,-0.360128,-0.147354,0.001552,0.708227,0.0


In [405]:
x_test1

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,...,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW,dia_semana,mes,anio
182607,1.410456,-0.986678,-0.445292,1.956608,1.425253,1.487843,-0.092076,-0.630817,2.188454,-0.304671,...,1.299191,-0.980643,-0.078603,-1.989900,-0.118318,-1.327161,-0.147354,-1.500212,0.708227,0.0
43245,-0.987836,-0.564650,1.002102,-0.379337,-0.988046,-0.989828,-0.092076,1.585245,-0.456944,-0.304671,...,0.245908,-0.980643,-0.078603,-0.662409,-0.118318,-0.843644,-0.147354,1.503315,-1.411977,0.0
43445,-0.984394,-0.597805,-0.974860,0.032455,-0.986069,-0.989828,-0.092076,1.585245,-0.456944,-0.304671,...,-0.299926,0.977715,-0.078603,0.001337,-0.118318,0.244269,-0.147354,1.503315,-1.411977,0.0
32465,-1.173349,0.346403,0.010157,-0.931405,-1.165191,-1.155006,-0.092076,1.585245,-0.456944,-0.849046,...,-1.325486,0.977715,6.908787,1.992574,0.887682,0.969544,0.285027,1.002727,-1.411977,0.0
50746,-0.858750,-0.552908,-0.237482,-0.587135,-0.844196,-0.824650,-0.092076,-0.630817,-0.456944,0.784079,...,-1.229322,-0.980643,-0.078603,-0.330536,-0.118318,0.365148,-0.147354,-1.500212,-1.411977,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52997,-0.820013,-0.161273,0.577131,-0.531500,-0.821139,-0.824650,-0.092076,-0.630817,-0.456944,-0.849046,...,-0.820902,1.956894,-0.078603,0.001337,-0.118318,-0.601886,-0.147354,-1.500212,-1.411977,0.0
81160,-0.335353,1.567588,-0.676307,-1.043625,-0.333696,-0.329115,-0.092076,-0.630817,-0.456944,-1.393421,...,-1.586257,1.956894,-0.078603,1.328828,-0.118318,1.694819,-0.147354,0.001552,0.708227,0.0
70746,-0.514569,-1.098573,1.596437,-0.239537,-0.506555,-0.494293,-0.092076,-0.630817,-0.456944,-0.849046,...,-0.805679,-0.001464,-0.078603,0.996956,-0.118318,0.727785,0.309048,-0.499036,0.708227,0.0
165770,1.120707,-0.605403,0.479807,0.258085,1.135042,1.157487,-0.092076,1.585245,-0.456944,0.784079,...,0.587249,0.977715,-0.078603,0.001337,-0.118318,-0.964524,-0.147354,1.002727,0.708227,0.0


In [406]:
model = DecisionTreeClassifier()

In [407]:
model.fit(x_train1, y_train1)

DecisionTreeClassifier()

In [408]:
predict1 = model.predict(x_test1)

In [409]:
predict1

array([0, 0, 0, ..., 0, 0, 0])

In [410]:
y_test1

182607    0
43245     0
43445     0
32465     0
50746     0
         ..
52997     0
81160     0
70746     0
165770    0
188254    0
Name: TX_FRAUD, Length: 60389, dtype: int64

### Modelo 2

El segundo modelo tendrá el árbol de decisiones con un límite en al altura de 5.

In [411]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(df_copia_std, y, test_size = 0.3, random_state = 42)
model = DecisionTreeClassifier(max_depth=5)
model.fit(x_train2, y_train2)
predict2 = model.predict(x_test2)
predict2

array([0, 0, 0, ..., 0, 0, 0])

### Modelo 3

El tercer modelo tendrá el árbol de decisiones sin límite en la altura, pero con un conjunto de datos de testeo del 50%.

In [412]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(df_copia_std, y, test_size = 0.5, random_state = 42)
model = DecisionTreeClassifier()
model.fit(x_train3, y_train3)
predict3 = model.predict(x_test3)
predict3

array([0, 0, 0, ..., 0, 0, 0])

### Modelo 4

El cuarto modelo tendrá el árbol de decisiones con un límite en la altura de 5 y con un conjunto de datos de testeo del 50%.

In [413]:
x_train4, x_test4, y_train4, y_test4 = train_test_split(df_copia_std, y, test_size = 0.5, random_state = 42)
model = DecisionTreeClassifier(max_depth=5)
model.fit(x_train4, y_train4)
predict4 = model.predict(x_test4)
predict4

array([0, 0, 0, ..., 0, 0, 0])

# 3) Validación del Modelo

Se usan las distintas métricas que tienen los árboles de decisiones para evaluar y validar el modelo

### Modelo 1

In [414]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test1, predict1)

array([[59887,     0],
       [    0,   502]])

La matriz de confusión tiene 59887 Verdaderos Positivos, 0 Falsos Positivos, 0 Falsos Negativos y 502 Verdaderos Negativos.

In [415]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test1, predict1)

1.0

La exactitud da con una puntuación de 1.

In [416]:
from sklearn.metrics import precision_score
precision_score(y_test1, predict1)

1.0

La precisión da con una puntuación de 1.

In [417]:
from sklearn.metrics import classification_report
print(classification_report(y_test1, predict1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59887
           1       1.00      1.00      1.00       502

    accuracy                           1.00     60389
   macro avg       1.00      1.00      1.00     60389
weighted avg       1.00      1.00      1.00     60389



Una vista general de las métricas que evaluan la validez del modelo. Se puede ver como utilizando un 70% de datos como datos de entrenamiento y 30% como datos de testeo, se obtiene un modelo bastante preciso.

### Modelo 2

In [418]:
confusion_matrix(y_test2, predict2)

array([[59887,     0],
       [    0,   502]])

In [419]:
accuracy_score(y_test2, predict2)

1.0

In [420]:
precision_score(y_test2, predict2)

1.0

In [421]:
print(classification_report(y_test2, predict2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59887
           1       1.00      1.00      1.00       502

    accuracy                           1.00     60389
   macro avg       1.00      1.00      1.00     60389
weighted avg       1.00      1.00      1.00     60389



Lo resultados son los mismos que en el modelo 1, a pesar de poner un límite en la altura del árbol de decisión.

### Modelo 3

In [422]:
confusion_matrix(y_test3, predict3)

array([[99788,     0],
       [    0,   860]])

La matriz de confusión tiene 99788 Verdaderos Positivos, 0 Falsos Positivos, 0 Falsos Negativos y 860 Verdaderos Negativos.

In [423]:
accuracy_score(y_test3, predict3)

1.0

La exactitud da con una puntuación de 1.

In [424]:
precision_score(y_test3, predict3)

1.0

La precisión da con una puntuación de 1.

In [425]:
print(classification_report(y_test3, predict3))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99788
           1       1.00      1.00      1.00       860

    accuracy                           1.00    100648
   macro avg       1.00      1.00      1.00    100648
weighted avg       1.00      1.00      1.00    100648



Utilizando un 50% de datos como testeo, también se obtiene un modelo de datos muy preciso.

### Modelo 4

In [426]:
confusion_matrix(y_test4, predict4)

array([[99788,     0],
       [    0,   860]])

In [427]:
accuracy_score(y_test4, predict4)

1.0

In [428]:
precision_score(y_test4, predict4)

1.0

In [429]:
print(classification_report(y_test4, predict4))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     99788
           1       1.00      1.00      1.00       860

    accuracy                           1.00    100648
   macro avg       1.00      1.00      1.00    100648
weighted avg       1.00      1.00      1.00    100648



Se obtienen los mismos resultados que en el modelo 3.

## Conclusiones

Como conclusión se puede decir que, incluso al variar el tamaño de los datos de entrenamiento o poner un límite en la altura del árbol de decisión, se obtienen modelos predictivos bastante precisos, en base a lo que se puede ver de las métricas de evaluación.